<a href="https://colab.research.google.com/github/datadigger01/AI-Trade/blob/main/uncomtrade_apicall_total.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade comtradeapicall

In [3]:
import pandas
import requests
import comtradeapicall

#set some variables
from datetime import date
from datetime import timedelta
today = date.today()
yesterday = today - timedelta(days=1)
lastweek = today - timedelta(days=7)

In [4]:
"""
UN Comtrade API를 이용한 다년도 데이터 수집 및 병합
2014년부터 2024년까지 연도별 12개월 데이터 수집
"""

import pandas as pd
import time
from datetime import datetime

# comtradeapicall 모듈이 있다고 가정
# import comtradeapicall

print("=" * 80)
print("UN Comtrade 다년도 데이터 수집 시작")
print("=" * 80)

# API 설정
subscription_key = '#################################'  # 자신의 키로 교체 필요

# 고정 파라미터
typeCode = 'C'
freqCode = 'M'
clCode = 'HS'
reporterCode = '156'  # China
cmdCode = '8415'  # 에어컨
flowCode = 'X'  # Export
partnerCode = '48,414,512,634,682,784'  # GCC countries  # 48:Bahrain, 414:Kuwait, 512:Oman, 634:Qatar, 682:Saudi Arabia, 784:United Arab Emirates
partner2Code = None
customsCode = None
motCode = None
maxRecords = 10000
format_output = 'JSON'
aggregateBy = None
breakdownMode = 'classic'
countOnly = None
includeDesc = True

# 데이터 수집 연도 범위
start_year = 2015
end_year = 2024

# 데이터프레임 리스트 (각 연도별 데이터 저장)
dataframes_list = []

# ============================================================================
# 연도별 데이터 수집
# ============================================================================
print(f"\n데이터 수집 기간: {start_year}년 ~ {end_year}년")
print(f"총 {end_year - start_year + 1}개 연도\n")

for year in range(start_year, end_year + 1):
    print(f"\n{'='*80}")
    print(f"[{year}년] 데이터 수집 중...")
    print(f"{'='*80}")

    # 해당 연도의 12개월 period 문자열 생성
    # 예: 2014년 -> '201401,201402,201403,...,201412'
    months = [f"{year}{str(month).zfill(2)}" for month in range(1, 13)]
    period_string = ','.join(months)

    print(f"Period: {period_string}")

    try:
        # API 호출
        df_year = comtradeapicall.getFinalData(
            subscription_key=subscription_key,
            typeCode=typeCode,
            freqCode=freqCode,
            clCode=clCode,
            period=period_string,
            reporterCode=reporterCode,
            cmdCode=cmdCode,
            flowCode=flowCode,
            partnerCode=partnerCode,
            partner2Code=partner2Code,
            customsCode=customsCode,
            motCode=motCode,
            maxRecords=maxRecords,
            format_output=format_output,
            aggregateBy=aggregateBy,
            breakdownMode=breakdownMode,
            countOnly=countOnly,
            includeDesc=includeDesc
        )

        # 데이터 수집 성공
        if df_year is not None and len(df_year) > 0:
            # 연도 컬럼 추가 (나중에 필터링할 때 유용)
            df_year['collection_year'] = year

            dataframes_list.append(df_year)
            print(f"✓ {year}년 데이터 수집 완료: {len(df_year)} rows")

            # 간단한 통계 출력
            if 'period' in df_year.columns:
                print(f"  수집된 기간: {df_year['period'].min()} ~ {df_year['period'].max()}")
            if 'primaryValue' in df_year.columns:
                print(f"  총 교역액: ${df_year['primaryValue'].sum():,.0f}")
        else:
            print(f"⚠ {year}년 데이터가 비어있습니다.")

        # API 호출 제한을 고려하여 대기 (필요시)
        # Comtrade API는 무료 계정 기준 시간당 호출 횟수 제한이 있음
        print(f"대기 중... (API 제한 고려)")
        time.sleep(2)  # 2초 대기

    except Exception as e:
        print(f"❌ {year}년 데이터 수집 실패: {str(e)}")
        continue

# ============================================================================
# 데이터 병합
# ============================================================================
print(f"\n{'='*80}")
print("데이터 병합 작업 시작")
print(f"{'='*80}")

if len(dataframes_list) > 0:
    print(f"\n수집된 데이터셋 개수: {len(dataframes_list)}개")

    # 모든 데이터프레임을 하나로 합치기
    combined_df = pd.concat(dataframes_list, ignore_index=True)

    print(f"\n병합 완료!")
    print(f"  - 총 행 수: {len(combined_df):,}")
    print(f"  - 총 열 수: {len(combined_df.columns)}")

    # 기본 정보 출력
    if 'period' in combined_df.columns:
        print(f"\n수집된 기간 범위:")
        print(f"  - 최소: {combined_df['period'].min()}")
        print(f"  - 최대: {combined_df['period'].max()}")

    if 'collection_year' in combined_df.columns:
        print(f"\n연도별 데이터 분포:")
        year_counts = combined_df['collection_year'].value_counts().sort_index()
        for year, count in year_counts.items():
            print(f"  - {year}년: {count:,} rows")

    # 데이터 저장
    output_filename = f'/content/comtrade_data_{start_year}_{end_year}.csv'
    combined_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"\n✓ 병합된 데이터를 '{output_filename}'에 저장했습니다.")

    # 기본 통계 출력
    print(f"\n{'='*80}")
    print("데이터 요약 통계")
    print(f"{'='*80}")

    if 'primaryValue' in combined_df.columns:
        print(f"\n교역액 통계:")
        print(f"  - 총 교역액: ${combined_df['primaryValue'].sum():,.0f}")
        print(f"  - 평균 교역액: ${combined_df['primaryValue'].mean():,.0f}")
        print(f"  - 최대 교역액: ${combined_df['primaryValue'].max():,.0f}")
        print(f"  - 최소 교역액: ${combined_df['primaryValue'].min():,.0f}")

    # 상위 5개 기간 출력
    if 'period' in combined_df.columns and 'primaryValue' in combined_df.columns:
        print(f"\n교역액 상위 5개 기간:")
        top_periods = combined_df.groupby('period')['primaryValue'].sum().sort_values(ascending=False).head()
        for period, value in top_periods.items():
            print(f"  - {period}: ${value:,.0f}")

    print(f"\n{'='*80}")
    print("✅ 모든 작업 완료!")
    print(f"{'='*80}")

else:
    print("\n⚠ 수집된 데이터가 없습니다.")

# ============================================================================
# 개별 연도 데이터도 저장 (선택사항)
# ============================================================================
save_individual_files = False  # True로 변경하면 연도별 파일도 저장

if save_individual_files and len(dataframes_list) > 0:
    print(f"\n{'='*80}")
    print("개별 연도 파일 저장 중...")
    print(f"{'='*80}")

    for i, df in enumerate(dataframes_list):
        year = start_year + i
        if 'collection_year' in df.columns:
            year = df['collection_year'].iloc[0]

        individual_filename = f'/contens/comtrade_data_{year}.csv'
        df.to_csv(individual_filename, index=False, encoding='utf-8-sig')
        print(f"✓ {year}년 데이터를 '{individual_filename}'에 저장했습니다.")

UN Comtrade 다년도 데이터 수집 시작

데이터 수집 기간: 2015년 ~ 2024년
총 10개 연도


[2015년] 데이터 수집 중...
Period: 201501,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,201512
⚠ 2015년 데이터가 비어있습니다.
대기 중... (API 제한 고려)

[2016년] 데이터 수집 중...
Period: 201601,201602,201603,201604,201605,201606,201607,201608,201609,201610,201611,201612
✓ 2016년 데이터 수집 완료: 72 rows
  수집된 기간: 201601 ~ 201612
  총 교역액: $1,091,552,479
대기 중... (API 제한 고려)

[2017년] 데이터 수집 중...
Period: 201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712
✓ 2017년 데이터 수집 완료: 72 rows
  수집된 기간: 201701 ~ 201712
  총 교역액: $1,289,591,855
대기 중... (API 제한 고려)

[2018년] 데이터 수집 중...
Period: 201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812
✓ 2018년 데이터 수집 완료: 72 rows
  수집된 기간: 201801 ~ 201812
  총 교역액: $1,212,522,048
대기 중... (API 제한 고려)

[2019년] 데이터 수집 중...
Period: 201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912
✓ 2019년 데이터 수집 완료: 72 rows
  수집된 기간: 2019

/tmp/ipython-input-2675095686.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dataframes_list, ignore_index=True)
